In [1]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

### Determining food names and tables

In [3]:
root = "../raw/blia.it"
foods = [food.split(".csv")[0] for food in os.listdir(root) if food.endswith(".csv")]

### Merging csv files and normalizing columns and unit

In [4]:
from IPython.display import display

initial_index_name = "valori"
final_index_name = "name"
units_column = "u.m."
value_column = "quantità"
factors = {
    "mg":1e-3,
    "mcg":1e-6
}
blia = pd.DataFrame([])
blia.index.name=final_index_name
for food in tqdm(foods):
    path = "{root}/{food}.csv".format(root=root, food=food)
    df = pd.read_csv(path, index_col=0, usecols=[0, 1, 2, 3])
    df = df.rename(columns=df.iloc[0]).drop(0)
    df.columns = [c.strip().lower() for c in [initial_index_name, *df.columns[1:]]]
    df.index = df[initial_index_name]
    df = df.drop(columns=[initial_index_name])
    for index, (unit, value) in df.iterrows():
        if unit in ["mg", "mcg"]:
            df.loc[index] = "g", float(value)*factors[unit]
        else:
            df.loc[index][value_column] = float(value)
        df = df.rename(index={index: "{index} | {unit}".format(index=index.lower(), unit=df.loc[index][units_column])})
    del df.index.name
    df = df.drop(columns=[units_column]).transpose()
    df.index = [food]
    df.index.name = final_index_name
    blia = blia.append(df)

blia.to_csv("../csv/blia.csv")